In [1]:
!pip install pytesseract

  Created wheel for pytesseract: filename=pytesseract-0.3.4-py2.py3-none-any.whl size=13431 sha256=edcc151117abf5473a1b76375d13cbfc094d96d9c1f22abf9c19ab615d2517f9
  Stored in directory: /root/.cache/pip/wheels/63/2a/a0/7596d2e0a73cf0aeffd6f6170862c4e73f3763b7827e48691a
Successfully built pytesseract


In [2]:
!pip install tesseract

     |████████████████████████████████| 45.6MB 91kB/s 
  Created wheel for tesseract: filename=tesseract-0.1.3-cp36-none-any.whl size=45562571 sha256=b6d95b555b2a015f8a49ba21f8815125e501967d5ca7eab06c2edf0328988c99
  Stored in directory: /root/.cache/pip/wheels/82/1f/d9/24797b123379e4ea9511cf660835468b62dad609634cad2aba
Successfully built tesseract


In [3]:
!pip install Pillow

In [4]:
!sudo apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,222 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [5]:
!sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
pkg-config is already the newest version (0.29.1-0ubuntu2).
python3-dev is already the newest version (3.6.7-1~18.04).
The following additional packages will be installed:
  libpoppler-cpp0v5 libpoppler-dev
The following NEW packages will be installed:
  libpoppler-cpp-dev libpoppler-cpp0v5 libpoppler-dev
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 41.3 kB of archives.
After this operation, 233 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpoppler-cpp0v5 amd64 0.62.0-2ubuntu2.10 [28.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpoppler-dev amd64 0.62.0-2ubuntu2.10 [4,608 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpoppler-cpp-dev amd64 0.62.0-2ubuntu2.10 [8,684 B]
Fetched 41.3 kB in 

In [6]:
!pip install pdftotext

     |████████████████████████████████| 122kB 3.5MB/s 
  Created wheel for pdftotext: filename=pdftotext-2.1.4-cp36-cp36m-linux_x86_64.whl size=54080 sha256=0dd58135f29ea1d2fdeb0589a7cb16234a3a8cd7f719081afc1cc223e0cffc8d
  Stored in directory: /root/.cache/pip/wheels/88/c3/62/c1ef7f738f8c0c1ac67cbe63786cae734d555c5077088036f0
Successfully built pdftotext


In [0]:
import pdftotext
import re
import pandas as pd
import os
import glob
import json

# OCR Framework to covert image to text

In [0]:
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract

def ocr_core(filename):
    """
    This function will handle the core OCR processing of images.
    """
    text = pytesseract.image_to_string(Image.open(filename))  
    return text



In [0]:

def pdf_core(filename):
    
    with open(filename, "rb") as f:
        pdf = pdftotext.PDF(f)
    return "\n\n".join(pdf)
    



    

# Create NER TAGS


In [0]:
def extension_type_process(extension,f):
  if extension == 'pdf':
    return pdf_core(f)
  elif extension == 'png':
    return ocr_core(f)

In [0]:
def find_start_end(labelcontent,text,label,category):
  annolist = []

  
  if label == 'CATEGORY':
    if category == 'Merrilllynch_CRA':
      text = 'Merrill Lynch'
    elif category == 'Schwab_IRA':
      text = 'Schwab'
    for match in re.finditer(text, labelcontent):
      annolist.append({"label":[label],"points":[{"start":match.start(),"end":match.end(),"text":text}]})
  elif label == 'PHONE':
    num = "("+text[0:3]+") "+text[3:6]+"-"+text[6:10]
    for match in re.finditer(r'\(\w{3}\) \w{3}-\w{4}', labelcontent):
      if labelcontent[match.start():match.end()] == num:
        annolist.append({"label":[label],"points":[{"start":match.start(),"end":match.end(),"text":num}]})
  elif label == 'CITY':
    city = text.split(',')[0]
    for match in re.finditer(city, labelcontent):
      annolist.append({"label":[label],"points":[{"start":match.start(),"end":match.end(),"text":city}]})
  elif label == 'EMAIL':
    for match in re.finditer(r'[\w\.-]+@[\w\.-]+', labelcontent):
      if labelcontent[match.start():match.end()] == text:
        annolist.append({"label":[label],"points":[{"start":match.start(),"end":match.end(),"text":text}]})
  else:
    for match in re.finditer(text, labelcontent):
      annolist.append({"label":[label],"points":[{"start":match.start(),"end":match.end(),"text":text}]})
  return annolist

In [0]:
def pdf_tag(file,category,filename,df):
    if category == 'Ameritrade Institutional':
      labels = ['CATEGORY','TITLE','CLIENT','ADDRESS','SOCIAL SECURITY NO','RELATIONSHIP NO.','ACC NO','ACC TYPE','PHONE']
    else:
      labels = ['CATEGORY','TITLE','CLIENT','ADDRESS','SOCIAL SECURITY NO','RELATIONSHIP NO.','ACC NO','ACC TYPE','PHONE','CITY','EMAIL']
    dict={}
    annolist = []
    content = pdf_core(file)
    
    dict['content'] = content
    for label in labels:
      result = df[(df['CATEGORY']==category) & (df['FILE NAME']==filename)][label]
      if (list(result) == []):
        continue
      else:
        text=str(result.values[0]).strip()
        annolist = annolist + find_start_end(content,text,label,category)
     
    dict['annotation'] = annolist
    
    return json.dumps(dict)


In [0]:
def png_tag(file,category,filename,df):
    labels = ['CATEGORY','TITLE','CLIENT','ADDRESS','SOCIAL SECURITY NO','RELATIONSHIP NO.','ACC NO','ACC TYPE','PHONE','CITY','EMAIL']
    dict={}
    annolist = []
    content = ocr_core(file)
    dict['content'] = content
    for label in labels:
      result = df[(df['CATEGORY']==category) & (df['FILE NAME']==filename)][label]
      if (list(result) == []):
        continue
      else:
        text=str(result.values[0]).strip()
        annolist = annolist + find_start_end(content,text,label,category)
    dict['annotation'] = annolist
    
    return json.dumps(dict)


In [0]:
df = pd.read_csv("/content/drive/My Drive/DOC-NER/IRA.csv")

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
categories = ['Merrilllynch_CRA','Schwab_IRA','Ameritrade Institutional']
pdf_doc_dict={}
png_doc_dict={}
for company in categories:
  fullpath = "/content/drive/My Drive/DOC-NER/"+company
  os.chdir(fullpath)
  all_filenames = [i for i in glob.glob('*.{}'.format('pdf'))]
  pdf_doc_dict[company] = [pdf_tag(f,company,f.split('.')[0],df) for f in all_filenames ]
  all_filenames = [i for i in glob.glob('*.{}'.format('png'))]
  png_doc_dict[company] = [png_tag(f,company,f.split('.')[0],df) for f in all_filenames ]



In [18]:
list(png_doc_dict.values())[0]

['{"content": "DTC8862\\nACATO671\\n\\nClient Account Transfer Form\\n\\nUse this form to transfer all or a portion of your account from\\nanother financial institution to your Merrill Lynch, Pierce, Fenner &\\nSmith Incorporated (\\u201cMerrill Lynch\\u201d) account.\\n\\nMERRILL bos\\n\\nA BANK OF AMERICA COMPANY\\n\\nFor best results, please complete the information requested below in parts 1 to 4 using ALL CAPITAL letters and\\nblack ink. If you need more room for information or signatures, use a copy of the relevant page or a blank sheet. Use\\none form for each account that you\'re transferring.\\n\\nThings to know before you begin:\\n\\nMany firms transfer assets using an automated transfer process, which typically takes 5 to 6 business days. Other\\nfirms either don\\u2019t use the automated process or must process some assets manually, which takes more time. The\\nfollowing estimated time frames for manual transfers are from the date of receipt of instructions by the deliverin

In [0]:
os.chdir("/content/drive/My Drive/DOC-NER/")
with open('testfile.txt', 'w') as filehandle:
  for category, tagtext in png_doc_dict.items():
    if len(tagtext) != 0:
      for listitem in tagtext:
        filehandle.write('%s\n' % listitem)
  for category, tagtext in pdf_doc_dict.items():
    if len(tagtext) != 0:
      for listitem in tagtext:
        filehandle.write('%s\n' % listitem)        

# Model

In [0]:

import json
import random
import logging
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None


In [21]:

import spacy
def train_spacy():

    TRAIN_DATA = convert_dataturks_to_spacy("/content/drive/My Drive/DOC-NER/train.txt")
    nlp = spacy.blank('en')  
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  
                    [annotations],  
                    drop=0.2,  
                    sgd=optimizer,  
                    losses=losses)
            print(losses)
    return nlp

nlp = train_spacy()

Statring iteration 0
{'ner': 26413.56943010138}
Statring iteration 1
{'ner': 1478.9695093278608}
Statring iteration 2
{'ner': 608.0166992206741}
Statring iteration 3
{'ner': 401.16371179629135}
Statring iteration 4
{'ner': 174.06746845084083}
Statring iteration 5
{'ner': 280.2045127802031}
Statring iteration 6
{'ner': 133.61405469105938}
Statring iteration 7
{'ner': 74.07527980328359}
Statring iteration 8
{'ner': 112.31061971597391}
Statring iteration 9
{'ner': 69.18309214731642}


In [22]:
    examples = convert_dataturks_to_spacy("/content/drive/My Drive/DOC-NER/validate.txt")
    tp=0
    tr=0
    tf=0

    ta=0
    c=0        
    for text,annot in examples:

        f=open("doc"+str(c)+".txt","w")
        doc_to_test=nlp(text)
        
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[]
        for ent in doc_to_test.ents:
            d[ent.label_].append(ent.text)

        for i in set(d.keys()):
            f.write("\n\n")
            f.write(i +":"+"\n")
            for j in set(d[i]):
                f.write(j.replace('\n','')+"\n")
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[0,0,0,0,0,0]
        for ent in doc_to_test.ents:
            doc_gold_text= nlp.make_doc(text)
            gold = GoldParse(doc_gold_text, entities=annot.get("entities"))
            y_true = [ent.label_ if ent.label_ in x else 'Not '+ent.label_ for x in gold.ner]
            y_pred = [x.ent_type_ if x.ent_type_ ==ent.label_ else 'Not '+ent.label_ for x in doc_to_test]  
            if(d[ent.label_][0]==0):
                
                (p,r,f,s)= precision_recall_fscore_support(y_true,y_pred,average='weighted')
                a=accuracy_score(y_true,y_pred)
                d[ent.label_][0]=1
                d[ent.label_][1]+=p
                d[ent.label_][2]+=r
                d[ent.label_][3]+=f
                d[ent.label_][4]+=a
                d[ent.label_][5]+=1
        c+=1
    
    for i in d:
        print("\n For Entity "+i+"\n")
        print("Accuracy : "+str((d[i][4]/d[i][5])*100)+"%")
        print("Precision : "+str(d[i][1]/d[i][5]))
        print("Recall : "+str(d[i][2]/d[i][5]))
        print("F-score : "+str(d[i][3]/d[i][5]))


 For Entity ACC TYPE

Accuracy : 99.96575928779319%
Precision : 0.9997554234842371
Recall : 0.9996575928779319
F-score : 0.9996860974639721

 For Entity CATEGORY

Accuracy : 98.65319865319864%
Precision : 0.9961056346598516
Recall : 0.9865319865319865
F-score : 0.9901992229391459

 For Entity TITLE

Accuracy : 100.0%
Precision : 1.0
Recall : 1.0
F-score : 1.0


In [33]:
def predict(filepath,type,output_filename,nlp):
        f=open(output_filename,"w")
        doc_to_test=""
        if type=='pdf':
          doc_to_test=nlp(pdf_core(filepath))
        elif type=='png':
          doc_to_test=nlp(ocr_core(filepath))
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[]
        for ent in doc_to_test.ents:
            d[ent.label_].append(ent.text)

        for i in set(d.keys()):
            print("\n\n")
            print(i +":"+"\n")
            for j in set(d[i]):
                print(j.replace('\n','')+"\n")

predict('/content/drive/My Drive/DOC-NER/Schwab_IRA/Bobby_J_Cole_.pdf','pdf','predict.txt',nlp)




ACC TYPE:

IRA     

IRA)

IRA.

IRA;

IRA to

IRA		

IRAs




TITLE:

Account Application.




CATEGORY:

Schwab representatives

Schwab and

Schwab Advisor

Schwab    

Schwab deems

Schwab:

Schwab                            

Schwab on

Schwab reserves

Schwab are

Schwab Qualified

Schwab is

Schwab”

Schwab (

Schwab any

Schwab)

Schwab.

Schwab research

Schwab harmless

Schwab                     

Schwab account

Schwab’s

Schwab to

Schwab 403(b)(7

Schwab or

Schwab changes

Schwab fails

Schwab,

Schwab

Schwab Corporation

Schwab Traditional

Schwab Bank

Schwab based

Schwab disclose

Schwab site

Schwab Individual

Schwab One

Schwab                                                         

Schwab supplemental

Schwab                         

Schwab Alliance

Schwab           

Schwab may

Schwab not

Schwab IRA

Schwab in

Schwab can

Schwab if

Schwab has

Schwab          

Schwab                

Schwab shall

Schwab                                               

In [36]:
def predict(filepath,type,output_filename,nlp):
        f=open(output_filename,"w")
        doc_to_test=""
        if type=='pdf':
          doc_to_test=nlp(pdf_core(filepath))
        elif type=='png':
          doc_to_test=nlp(ocr_core(filepath))
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[]
        for ent in doc_to_test.ents:
            d[ent.label_].append(ent.text)

        for i in set(d.keys()):
            print("\n\n")
            print(i +":"+"\n")
            for j in set(d[i]):
                print(j.replace('\n','')+"\n")

predict('/content/drive/My Drive/DOC-NER/Schwab_IRA/Donna_E_Warren_.pdf','pdf','predict.txt',nlp)




ACC TYPE:

IRA     

IRA)

IRA.

IRA;

IRA to

IRA		

IRAs




TITLE:

Account Application.




CATEGORY:

Schwab representatives

Schwab and

Schwab Advisor

Schwab    

Schwab deems

Schwab:

Schwab                            

Schwab on

Schwab reserves

Schwab are

Schwab Qualified

Schwab is

Schwab”

Schwab (

Schwab any

Schwab)

Schwab.

Schwab research

Schwab harmless

Schwab                     

Schwab account

Schwab’s

Schwab to

Schwab 403(b)(7

Schwab or

Schwab changes

Schwab fails

Schwab,

Schwab

Schwab Corporation

Schwab Traditional

Schwab Bank

Schwab based

Schwab disclose

Schwab site

Schwab Individual

Schwab One

Schwab                                                         

Schwab supplemental

Schwab                         

Schwab Alliance

Schwab           

Schwab may

Schwab not

Schwab IRA

Schwab in

Schwab can

Schwab if

Schwab has

Schwab          

Schwab                

Schwab shall

Schwab                                               

In [32]:
def predict(filepath,type,output_filename,nlp):
        f=open(output_filename,"w")
        doc_to_test=""
        if type=='pdf':
          doc_to_test=nlp(pdf_core(filepath))
        elif type=='png':
          doc_to_test=nlp(ocr_core(filepath))
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[]
        for ent in doc_to_test.ents:
            d[ent.label_].append(ent.text)

        for i in set(d.keys()):
            print("\n\n")
            print(i +":"+"\n")
            for j in set(d[i]):
                print(j.replace('\n','')+"\n")

predict('/content/drive/My Drive/DOC-NER/Merrilllynch_CRA/Candy T Lee.pdf','pdf','predict.txt',nlp)




CITY:

Pasadena,




ACC TYPE:

IRA,

IRA                                    




CATEGORY:

Merrill Lynch”

Merrill Lynch)

Merrill Lynch as

Merrill Lynch,

Merrill Lynch.

Merrill Lynch account

Merrill Lynch                            

Merrill Lynch accepts




SOCIAL SECURITY NO:

204842467

